In [1]:
# Import libraries
import os
import cv2
import numpy as np

# Define paths
base_dir = os.path.abspath('')
prototxt_path = os.path.join(base_dir, 'model_data', 'MobileNetSSD_deploy.prototxt.txt')
caffemodel_path = os.path.join(base_dir, 'model_data', 'MobileNetSSD_deploy.caffemodel')

# Read the model
model = cv2.dnn.readNetFromCaffe(prototxt_path, caffemodel_path)

# initialize the list of class labels MobileNet SSD was trained to
# detect, then generate a set of bounding box colors for each class
CLASSES = ["background", "aeroplane", "bicycle", "bird", "boat",
           "bottle", "bus", "car", "cat", "chair", "cow", "diningtable",
           "dog", "horse", "motorbike", "person", "pottedplant", "sheep",
           "sofa", "train", "tvmonitor"]
COLORS = np.random.uniform(0, 255, size=(len(CLASSES), 3))


In [2]:
# Create directory 'updated_images' if it does not exist
if not os.path.exists('updated_images'):
    print("New directory created")
    os.makedirs('updated_images')

# Loop through all images and save images with marked objects
for file in os.listdir(os.path.join(base_dir, 'images')):
    file_name, file_extension = os.path.splitext(file)
    if (file_extension in ['.png','.jpg']):
        print("Image path: {}".format(os.path.join(base_dir, 'images', file)))

        image = cv2.imread(os.path.join(base_dir, 'images', file))

        (h, w) = image.shape[:2]
        blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 0.007843, (300, 300), 127.5)

        model.setInput(blob)
        detections = model.forward()

        # Create frame around face
        for i in range(0, detections.shape[2]):

            confidence = detections[0, 0, i, 2]

            # If confidence > 0.2, show box around face
            if (confidence > 0.2):
                idx = int(detections[0, 0, i, 1])
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (startX, startY, endX, endY) = box.astype("int")
                
                label = "{}: {:.2f}%".format(CLASSES[idx], confidence * 100)
                print("[INFO] {}".format(label))
                cv2.rectangle(image, (startX, startY), (endX, endY), COLORS[idx], 2)
                y = startY - 15 if startY - 15 > 15 else startY + 15
                cv2.putText(image, label, (startX, y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLORS[idx], 2)
                

        cv2.imwrite(os.path.join(base_dir, 'updated_images', file), image)
        print("Image " + file + " converted successfully")

Image path: C:\UBS\Dev\neuralnet\neuralnet\FaceExtractor\images\cars.jpg
[INFO] car: 100.00%
[INFO] car: 100.00%
[INFO] car: 100.00%
[INFO] car: 99.96%
Image cars.jpg converted successfully
